In [2]:
import utils.spacetrack as strack
import utils.time as tu
import requests
import json
from datetime import datetime, timedelta
import time

In [ ]:
raise Exception("Are you sure you want to be running this notebook? Really it should only be ran one time. If yes you can comment this out")

## Establish connection to space-track.org API

You need an account to access this data programmatically, since they want to track who is using their api. They WILL ban your accoun (speaking from personal experience) if you query them too many times in a row, so it's important to access the API carefully when performing multiple calls in a row. 

My login information is in a file ./ted-login.ini, which will NOT be committed to the repository. Be careful to check their your login information not committed. 

In [4]:
CREDENTIALS_NEVER_PRINT = strack.loadSiteCred("./utils/ted-strack.ini")

In [5]:
with requests.Session() as session:
    login_resp = strack.api_login(session, CREDENTIALS_NEVER_PRINT)
    print("you logged in!")
    session.close()
print("you logged out!")

you logged in!
you logged out!


## Pull Catalog Snapshots

we need to do the following

1. establish a start date

2. establish an end date

3. write code that access the space-track.api (without violating its rules) and pull data 

4. save each day as its own text file 

*NOTE: this code should really only be ran one time, so I'm not too worried about optimizing/multi-threading*

In [ ]:
start_string = '2023-03-14'

stop_string = '2025-03-15'

n_pulls = 0

with requests.Session() as session:
    login_resp = strack.api_login(session, CREDENTIALS_NEVER_PRINT)

    start_date = datetime.strptime(start_string, '%Y-%m-%d')
    end_date = datetime.strptime(stop_string, '%Y-%m-%d')

    current_date = start_date

    while current_date <= end_date:
        print(current_date)
    
        next_date = current_date + timedelta(days=15)
        res = strack.api_catalog_snapshot(session, tu.timestamp_query(current_date), tu.timestamp_query(next_date))
        with open("../data/raw/test/snap_satcat_"+tu.timestampe_file(current_date)+".txt", 'w') as file:
            file.write(res.text)

        current_date = next_date

        n_pulls+=1

        if n_pulls > 10:
            print("Snoozing for 60 secs for rate limit reasons (max 20/min and 200/hr)...")
            time.sleep(60)
            n_pulls = 0

    session.close()

print("done!")


## State Catalogs: decayed and not decayed

We are also interested in getting state catalog information. Our objects should be in orbit, but we should account for any objects that decayed. Therefore, we have two catalogs: the active one and the decayed one

In [ ]:
new_catalogs = strack.request_repull_current_and_decayed_satcats("./utils/ted-strack.ini")

with open('../data/raw/current_satcat_'+tu.timestamp_file()+'.json', 'w') as json_file:
    json.dump(json.loads(new_catalogs[0]), json_file)

with open('../data/raw/decayed_satcat_'+tu.timestamp_file()+'.json', 'w') as json_file:
     json.dump(json.loads(new_catalogs[1]), json_file)